## ברק וורצברגר 316597541
## רתם בן עטר 318299914
## קישור לגיטהב
## https://github.com/barakwir/cars-prices-prediction.git

In [59]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os
from datetime import datetime

In [60]:
#פונקציה שעוצרת את הלולא ובודקת האם בעמוד הנוכחי יש חלון בו רשום האם בעמוד אין יותר מודעות
def all_pages(soup):
    eror = soup.find('div', class_="d-none p-1")
    if type(eror) == type(None):
        return False
    return True

In [61]:
#פונקציה שמחזירה רשימה עם כל המודעות בעמוד
def all_ads(soup):
    car_ads = soup.find('div', class_="cards-wrap s m l")
    ads = car_ads.find_all('div', class_="card-body p-md-3")
    return ads

In [62]:
#פונקציה שמקבלת טבלת נתונים מהמודעה ומחזירה מילון עם מילת מפתח שם הנתון והערך שהוזן עבורו באתר
def read_table(tables):
    ad_table_data = {}
    #מעבר על הטבלאות
    for table in tables:
        #למצוא את כל השורות בטבלה
        rows = table.find_all('tr')
                
        # לולאה על כל השורות בטבלה
        for row in rows:
            #למצוא את כל התאים בטבלה
            cells = row.find_all('td')
            #בדיקה אם אורך התאים 2 אז תא ראשון הוא שם העמודה ותא שני הוא הערך
            if len(cells) == 2:
                key = cells[0].get_text().strip()
                value = cells[1].get_text().strip()
                ad_table_data[key] = value
    return ad_table_data

In [63]:
#פונקציה שבודקת האם קיים ערך
def there_is_a_value(element):
    if element:
         value = element.get_text()
         return value
    else:
         value = None
         return value

In [64]:
#פונקציה שמפרידה בין תאריך יצירה לתאריך הקפצה
def split_div(soup):
    creation_date=soup[0]
    last_bump=soup[1]
    

    #my date formt 
    date_format = r'\d{2}/\d{2}/\d{4}'
    dates_dict = {}
    barak=re.findall(date_format, creation_date.get_text())
    dates_dict['תאריך יצירה'] = re.findall(date_format, creation_date.get_text())[0]
    dates_dict['תאריך הקפצה'] = re.findall(date_format,  last_bump.get_text())[0]
    
    return dates_dict

In [65]:
# רשימה אליה נכניס את כל הדאטה
ad_data = []
# כותרת ההודעה
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36",
    "Accept-Encoding": "gzip, deflate",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "DNT": "1",
    "Connection": "close",
    "Upgrade-Insecure-Requests": "1"
}
page_index=1
# לולאה על כל העמודים
while True:
    #יצירת קישור בצירוף מספר העמוד הנוכחי
    url = f'https://www.ad.co.il/car?keyword=פיאט&pageindex={page_index}'
    response= requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    the_last_page = all_pages(soup)
    # בדיקה האם קראנו את כל המודעות מכל העמודים
    if the_last_page:
        break
    page_index+=1
    #פונקציה שתחזיר ברשימה את כל המודעות בעמוד הנוכחי
    ads= all_ads(soup)
    # לולאה על כל המודעות בעמוד
    for ad in ads:
        # מציאת קישור למודעה
        ad_link = ad.find('a')['href']
        #יצירת קישור
        ad_url = f"https://www.ad.co.il{ad_link}"
        #קריאה לדף המודעה
        ad_response = requests.get(ad_url)
        ad_soup1 = BeautifulSoup(ad_response.content, 'html.parser')
        #משיכת שם היצרן והדגם
        ad_soup=ad_soup1.find('div',class_='tab-content bg-white rounded-3 border p-3 col-xxl-10 offset-xxl-1 col-md-12 offset-md-0')
        title = ad_soup.find('h2', class_='card-title').get_text()
        manufacturer=title.split()
        #מציאת המחיר במודעה
        price_element = ad.find('div', class_='price ms-1')
        #קריאה לפונקציה שבודקת ומחזירה את המחיר או נאן
        price=there_is_a_value(price_element)
        # מציאת טבלת הנתונים של הרכב
        tables = ad_soup.find_all('table', class_='table table-sm mb-4')
        # קבלה של מילון עם מפתחות שהם ראשי העמודות בדאטה פריים וערכים שהם הפרטים שהכניסו הלקוחות
        ad_table_data = read_table(tables)
        #צמצום של הHTML
        others=ad_soup.find('div', class_='order-first order-sm-0')
        #שליפת התיאור ובדיקה שורה אחרי שאינו ערך חסר
        description_element=others.find('p',class_='text-word-break')
        description=there_is_a_value(description_element)
        #קריאה של תאריכי פרסום והקפצה של המודעה
        ad_date_element1=others.find('div',class_='d-flex flex-row align-items-center justify-content-center flex-wrap')
        ad_date_element=ad_date_element1.find_all('div',class_='px-3')
        #קריאה לפונקציה לקבלת מילון עם תארוך העלה ותאריך הקפצה
        ad_date=split_div(ad_date_element)
        #שליפת קישורי התמונות
        image_soup=ad_soup.find('div',class_='col-xxl-8 col-lg-8 col-md-7')
        img_tags = image_soup.find_all('img')
        #מקבלים גם את התמונה המוגדלת אזי אם יש תמונות נוריד אותה מהספירה כי זה כפול אם לא נשאיר את הכמות 0 או 1
        if len(img_tags)>1:
            img_tags=len(img_tags)-1
        else:
            img_tags=len(img_tags)
        if img_tags==None:
            img_tags=0
        # הוספת המידע לרשימה
        ad_data.append({
            'יצרן': manufacturer[0],
            'דגם': manufacturer[1],
            'מחיר': price,
            **ad_table_data,  
            **ad_date,
            'תיאור': description,
            'מספר תמונות': img_tags
        })

# הפיכה לדאטה פריים
df = pd.DataFrame(ad_data)
#סידור העמודות בסדר המבוקש
new_order = ['יצרן', 'שנה', 'דגם', 'יד', 'ת. הילוכים', 'נפח', 'סוג מנוע', 'בעלות קודמת', 'בעלות נוכחית', 'אזור', 'עיר', 'מחיר','מספר תמונות', 'תאריך יצירה', 'תאריך הקפצה', 'תיאור',  'צבע', 'ק"מ','טסט עד',]
df = df[new_order]

#הצגה יפה של הדאטה
display(df)

,יצרן,שנה,דגם,יד,ת. הילוכים,נפח,סוג מנוע,בעלות קודמת,בעלות נוכחית,אזור,עיר,מחיר,מספר תמונות,תאריך יצירה,תאריך הקפצה,תיאור,צבע,"ק""מ",טסט עד
0,פיאט,2010,500,2,רובוטית,"1,400",בנזין,פרטית,פרטית,NaN,עין מאהל,"13,000 ₪",4,22/04/2024,22/04/2024,None,צהוב,"130,000",NaN
1,פיאט,2011,500,5,רובוטית,"1,200",בנזין,פרטית,פרטית,בקעת אונו,אור יהודה,"15,500 ₪",5,15/02/2024,15/02/2024,"פיאט 500 למכירה, רכב מפנק מאוד! נוח בחניות, חס...",צהוב,"134,000",NaN
2,פיאט,2015,500L,3,אוטומטית,"1,200",דיזל,פרטית,פרטית,תל אביב,תל אביב יפו,"22,500 ₪",7,18/12/2023,18/12/2023,"על הרכב\n\nיד שלישית\n\n1248 סמ""ק\n\nלזריזים ....",לבן,"210,000",NaN
3,פיאט,2013,500,3,אוטומטית,"1,200",בנזין,פרטית,פרטית,ירושלים והסביבה,ירושלים,"18,500 ₪",6,16/12/2023,16/12/2023,"פיאט 500 במצב מצויין!!! ללא תקלות, ללא תאונות....",לבן שנהב,"141,000",NaN
4,פיאט,2010,בראבו,4,רובוטית,"1,400",בנזין,פרטית,פרטית,חולון - בת ים,חולון,"13,000 ₪",8,04/12/2023,04/12/2023,למכירה פיאט בראבו 2010 סנסיישן 1.4 רובוטית. \n...,כחול מטאלי,"247,000",12/2024
5,פיאט,2013,500,4,אוטומטית,"1,200",בנזין,פרטית,פרטית,חולון - בת ים,חולון,"20,500 ₪",8,28/09/2023,28/09/2023,למכירה FIAT 500 דגם POP אוטומט מנוע 1200 \nשנת...,לבן פנינה,"106,000",NaN
6,פיאט,2016,קובו,1,אוטומטית,"1,200",דיזל,ליסינג,ליסינג,אשדוד - אשקלון,אשקלון,"25,000 ₪",4,24/04/2023,06/07/2023,"פיאט קובו דיזל 1.2 80 כ""ס אוט' החל מ- 450 ₪*.\...",לבן,"162,000",NaN
7,פיאט,2011,פנדה,4,ידנית,"1,300",בנזין,פרטית,פרטית,NaN,שילה,"6,500 ₪",3,14/06/2023,14/06/2023,חסכונית בטירוף לפני כמה ימים הוחלפו בולמים קדמ...,אפור,"230,000",NaN
8,פיאט,2016,קובו,1,אוטומטית,"1,200",דיזל,ליסינג,ליסינג,אשדוד - אשקלון,אשקלון,"27,990 ₪",5,24/04/2023,09/06/2023,"פיאט קובו דיזל 1.2 80 כ""ס אוט' החל מ-499 ₪*.\n...",לבן,"128,000",NaN
9,פיאט,2015,500,2,רובוטית,"1,200",בנזין,פרטית,פרטית,תל אביב,תל אביב יפו,"33,000 ₪",5,06/06/2023,06/06/2023,פיאט 500 2015 עם גג פנורמי\nשמורה מבפנים ומבחו...,שחור,"90,000",NaN


In [66]:
#שינוי שמות כל העמודות והטייפים של כל עמודה
df = df.rename(columns={'יצרן': 'manufactor'})
df['manufactor'] = df['manufactor'].astype(str)
df = df.rename(columns={'שנה': 'Year'})
df['Year'] = df['Year'].astype(int)
df = df.rename(columns={'דגם': 'model'})
df['model'] = df['model'].astype(str)
df = df.rename(columns={'יד': 'Hand'})
df['Hand'] = df['Hand'].astype(int)
df = df.rename(columns={'ת. הילוכים': 'Gear'})
df['Gear'] = pd.Categorical(df['Gear'])
df = df.rename(columns={'נפח': 'Engine_capacity'})
df['Engine_capacity'] = df['Engine_capacity'].str.replace(',', '').astype(int)
df = df.rename(columns={'סוג מנוע': 'Engine_type'})
df['Engine_type'] = pd.Categorical(df['Engine_type'])
df = df.rename(columns={'בעלות קודמת': 'Prev_ownership'})
df['Prev_ownership'] = pd.Categorical(df['Prev_ownership'])
df = df.rename(columns={'בעלות נוכחית': 'Curr_ownership'})
df['Curr_ownership'] = pd.Categorical(df['Curr_ownership'])
df = df.rename(columns={'אזור': 'Area'})
df['Area'] = df['Area'].astype(str)
df = df.rename(columns={'עיר': 'City'})
df['City'] = df['City'].astype(str)
df = df.rename(columns={'מחיר': 'Price'})
df['Price'] = df['Price'].str.replace('[₪,]', '', regex=True).astype(float)
df = df.rename(columns={'מספר תמונות': 'Pic_num'})
df['Pic_num'] = df['Pic_num'].astype(int)
df = df.rename(columns={'תאריך יצירה': 'Cre_date'})
df['Cre_date'] = df['Cre_date'].apply(convert_to_datetime)
df = df.rename(columns={'תאריך הקפצה': 'Repub_date'})
df['Repub_date'] = df['Repub_date'].apply(convert_to_datetime)
df = df.rename(columns={'תיאור': 'Description'})
df['Description'] = df['Description'].astype(str)
df = df.rename(columns={'צבע': 'Color'})
df['Color'] = df['Color'].astype(str)
df = df.rename(columns={'ק"מ': 'Km'})
df['Km'] = pd.to_numeric(df['Km'].str.replace(',', ''), errors='coerce').astype('Int64')
df = df.rename(columns={'טסט עד': 'Test'})
df['Test'] = pd.to_datetime(df['Test'], format='%m/%Y', errors='coerce')
df['Test'] = (df['Test'] - pd.Timestamp.today()).dt.days.astype('Int64')

print(df.dtypes)

manufactor                 object
Year                        int32
model                      object
Hand                        int32
Gear                     category
Engine_capacity             int32
Engine_type              category
Prev_ownership           category
Curr_ownership           category
Area                       object
City                       object
Price                     float64
Pic_num                     int32
Cre_date           datetime64[ns]
Repub_date         datetime64[ns]
Description                object
Color                      object
Km                          Int64
Test                        Int64
dtype: object


In [67]:
display(df)


,manufactor,Year,model,Hand,Gear,Engine_capacity,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test
0,פיאט,2010,500,2,רובוטית,1400,בנזין,פרטית,פרטית,nan,עין מאהל,13000.0,4,2024-04-22,2024-04-22,None,צהוב,130000,<NA>
1,פיאט,2011,500,5,רובוטית,1200,בנזין,פרטית,פרטית,בקעת אונו,אור יהודה,15500.0,5,2024-02-15,2024-02-15,"פיאט 500 למכירה, רכב מפנק מאוד! נוח בחניות, חס...",צהוב,134000,<NA>
2,פיאט,2015,500L,3,אוטומטית,1200,דיזל,פרטית,פרטית,תל אביב,תל אביב יפו,22500.0,7,2023-12-18,2023-12-18,"על הרכב\n\nיד שלישית\n\n1248 סמ""ק\n\nלזריזים ....",לבן,210000,<NA>
3,פיאט,2013,500,3,אוטומטית,1200,בנזין,פרטית,פרטית,ירושלים והסביבה,ירושלים,18500.0,6,2023-12-16,2023-12-16,"פיאט 500 במצב מצויין!!! ללא תקלות, ללא תאונות....",לבן שנהב,141000,<NA>
4,פיאט,2010,בראבו,4,רובוטית,1400,בנזין,פרטית,פרטית,חולון - בת ים,חולון,13000.0,8,2023-12-04,2023-12-04,למכירה פיאט בראבו 2010 סנסיישן 1.4 רובוטית. \n...,כחול מטאלי,247000,166
5,פיאט,2013,500,4,אוטומטית,1200,בנזין,פרטית,פרטית,חולון - בת ים,חולון,20500.0,8,2023-09-28,2023-09-28,למכירה FIAT 500 דגם POP אוטומט מנוע 1200 \nשנת...,לבן פנינה,106000,<NA>
6,פיאט,2016,קובו,1,אוטומטית,1200,דיזל,ליסינג,ליסינג,אשדוד - אשקלון,אשקלון,25000.0,4,2023-04-24,2023-07-06,"פיאט קובו דיזל 1.2 80 כ""ס אוט' החל מ- 450 ₪*.\...",לבן,162000,<NA>
7,פיאט,2011,פנדה,4,ידנית,1300,בנזין,פרטית,פרטית,nan,שילה,6500.0,3,2023-06-14,2023-06-14,חסכונית בטירוף לפני כמה ימים הוחלפו בולמים קדמ...,אפור,230000,<NA>
8,פיאט,2016,קובו,1,אוטומטית,1200,דיזל,ליסינג,ליסינג,אשדוד - אשקלון,אשקלון,27990.0,5,2023-04-24,2023-06-09,"פיאט קובו דיזל 1.2 80 כ""ס אוט' החל מ-499 ₪*.\n...",לבן,128000,<NA>
9,פיאט,2015,500,2,רובוטית,1200,בנזין,פרטית,פרטית,תל אביב,תל אביב יפו,33000.0,5,2023-06-06,2023-06-06,פיאט 500 2015 עם גג פנורמי\nשמורה מבפנים ומבחו...,שחור,90000,<NA>


In [68]:
#יצירת נתיב לתיקייה
folder_path = r"C:\Users\motiw\Desktop\תעשייה וניהול\כרייה וניתוח נתונים"
csv_path = os.path.join(folder_path, "ad_data.csv")
# שמירת הדאטהפריים כקובץ CSV
df.to_csv(csv_path, index=False, encoding='utf-8-sig')